# Install and Import Dependencies

In [2]:
%pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

2023-10-30 19:24:32.022878: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import uuid
import os
mp_hands = mp.solutions.hands

# Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [4]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

PoseLandmark.NOSE
PoseLandmark.LEFT_EYE_INNER
PoseLandmark.LEFT_EYE
PoseLandmark.LEFT_EYE_OUTER
PoseLandmark.RIGHT_EYE_INNER
PoseLandmark.RIGHT_EYE
PoseLandmark.RIGHT_EYE_OUTER
PoseLandmark.LEFT_EAR
PoseLandmark.RIGHT_EAR
PoseLandmark.MOUTH_LEFT
PoseLandmark.MOUTH_RIGHT
PoseLandmark.LEFT_SHOULDER
PoseLandmark.RIGHT_SHOULDER
PoseLandmark.LEFT_ELBOW
PoseLandmark.RIGHT_ELBOW
PoseLandmark.LEFT_WRIST
PoseLandmark.RIGHT_WRIST
PoseLandmark.LEFT_PINKY
PoseLandmark.RIGHT_PINKY
PoseLandmark.LEFT_INDEX
PoseLandmark.RIGHT_INDEX
PoseLandmark.LEFT_THUMB
PoseLandmark.RIGHT_THUMB
PoseLandmark.LEFT_HIP
PoseLandmark.RIGHT_HIP
PoseLandmark.LEFT_KNEE
PoseLandmark.RIGHT_KNEE
PoseLandmark.LEFT_ANKLE
PoseLandmark.RIGHT_ANKLE
PoseLandmark.LEFT_HEEL
PoseLandmark.RIGHT_HEEL
PoseLandmark.LEFT_FOOT_INDEX
PoseLandmark.RIGHT_FOOT_INDEX


<img src="https://i.imgur.com/qpRACer.png" style="height:300px" >

In [5]:
for ldmrk in mp_hands.HandLandmark:
    print(ldmrk)

HandLandmark.WRIST
HandLandmark.THUMB_CMC
HandLandmark.THUMB_MCP
HandLandmark.THUMB_IP
HandLandmark.THUMB_TIP
HandLandmark.INDEX_FINGER_MCP
HandLandmark.INDEX_FINGER_PIP
HandLandmark.INDEX_FINGER_DIP
HandLandmark.INDEX_FINGER_TIP
HandLandmark.MIDDLE_FINGER_MCP
HandLandmark.MIDDLE_FINGER_PIP
HandLandmark.MIDDLE_FINGER_DIP
HandLandmark.MIDDLE_FINGER_TIP
HandLandmark.RING_FINGER_MCP
HandLandmark.RING_FINGER_PIP
HandLandmark.RING_FINGER_DIP
HandLandmark.RING_FINGER_TIP
HandLandmark.PINKY_MCP
HandLandmark.PINKY_PIP
HandLandmark.PINKY_DIP
HandLandmark.PINKY_TIP


# Calculate Angles

In [5]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Pose Detection

In [6]:
# JUST THE HANDS
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()

        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detections
        print(results)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
            
        # Save our image    
        cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [3]:
# THE HANDS AND THE POSE
# TODO: REMOVE THE mini hands from the pose
cap = cv2.VideoCapture(0)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose, mp_hands.Hands(min_detection_confidence=0.3, min_tracking_confidence=0.3) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        # Get image used for pose estimation
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        hand_results = hands.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            hand_landmarks = hand_results.multi_hand_landmarks

            # PRINTING THE POSE COORDINATES
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            print("SHOULDERS")
            print("Left_Shoulder --- x: " + str(left_shoulder.x) + "y: " + str(left_shoulder.y) +  "z: " + str(left_shoulder.z))
            print("Right_Shoulder --- x: " + str(right_shoulder.x) + "y: " + str(right_shoulder.y) + "z: " + str(right_shoulder.z))

            print("ELBOWS")
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            print("Left_elbow --- x: " + str(left_elbow.x) + "y: " + str(left_elbow.y) + "z: " + str(left_elbow.z))
            print("Right_elbow --- x: " + str(right_elbow.x) + "y: " + str(right_elbow.y) + "z: " + str(right_elbow.z))

            # LEFT HAND
            left_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value]
            left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
            left_thumb = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            print("LEFT HAND")
            print("Left_pinky --- x: " + str(left_pinky.x) + "y: " + str(left_pinky.y) + "z: " + str(left_pinky.z))
            print("left_index --- x: " + str(left_index.x) + "y: " + str(left_index.y) + "z: " + str(left_index.z))
            print("left_thumb --- x: " + str(left_thumb.x) + "y: " + str(left_thumb.y) + "z: " + str(left_thumb.z))
            print("Left_wrist --- x: " + str(left_wrist.x) + "y: " + str(left_wrist.y) + "z: " + str(left_wrist.z))

            # RIGHT HAND
            right_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value]
            right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]
            right_thumb = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            print("RIGHT HAND")
            print("Right_pinky --- x: " + str(right_pinky.x) + "y: " + str(right_pinky.y) + "z: " + str(right_pinky.z))
            print("Right_index --- x: " + str(right_index.x) + "y: " + str(right_index.y) + "z: " + str(right_index.z))
            print("Right_thumb --- x: " + str(right_thumb.x) + "y: " + str(right_thumb.y) + "z: " + str(right_thumb.z))
            print("Right_wrist: x: " + str(right_wrist.x) + "y: " + str(right_wrist.y) + "z: " + str(right_wrist.z))

            # Lower body coordinates
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            print("LEFT KNEE --- x: " + str(left_knee.x) + " y: " + str(left_knee.y) + "z: " + str(left_knee.z))

            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            print("RIGHT KNEE --- x: " + str(right_knee.x) + " y: " + str(right_knee.y) + "z: " + str(right_knee.z))

            # LEFT FOOT
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
            left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
            print("LEFT FOOT")
            print("Left_ankle --- x: " + str(left_ankle.x) + " y: " + str(left_ankle.y) + " z: " + str(left_ankle.z))
            print("Left_heel --- x: " + str(left_heel.x) + " y: " + str(left_heel.y) + " z: " + str(left_heel.z))
            print("Left_foot_index --- x: " + str(left_foot_index.x) + " y: " + str(left_foot_index.y) + " z: " + str(left_foot_index.z))

            # RIGHT FOOT
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
            right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            print("RIGHT FOOT")
            print("Right_ankle --- x: " + str(right_ankle.x) + " y: " + str(right_ankle.y) + " z: " + str(right_ankle.z))
            print("Right_heel --- x: " + str(right_heel.x) + " y: " + str(right_heel.y) + " z: " + str(right_heel.z))
            print("Right_foot_index --- x: " + str(right_foot_index.x) + " y: " + str(right_foot_index.y) + " z: " + str(right_foot_index.z))

            frame_shape_0, frame_shape_1 = frame.shape[0], frame.shape[1]

            # DISPLAY AREA OF RIGHT FOOT ---------------
            right_foot_pts = np.array([
                [int(right_ankle.x * frame_shape_1), int(right_ankle.y * frame_shape_0)],
                [int(right_heel.x * frame_shape_1), int(right_heel.y * frame_shape_0)],
                [int(right_foot_index.x * frame_shape_1), int(right_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the right foot with a specific color
            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))

            # DISPLAY AREA OF LEFT FOOT ---------------
            left_foot_pts = np.array([
                [int(left_ankle.x * frame_shape_1), int(left_ankle.y * frame_shape_0)],
                [int(left_heel.x * frame_shape_1), int(left_heel.y * frame_shape_0)],
                [int(left_foot_index.x * frame_shape_1), int(left_foot_index.y * frame_shape_0)]
            ], np.int32).reshape((-1, 1, 2))

            # Fill the area of the left foot with a specific color
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))
    
            # DISPLAY AREA OF RIGHT HAND ---------------
            right_hand_pts = np.array([
                [int(right_pinky.x * frame_shape_1), int(right_pinky.y * frame_shape_0), right_pinky.z],
                [int(right_index.x * frame_shape_1), int(right_index.y * frame_shape_0), right_index.z],
                [int(right_thumb.x * frame_shape_1), int(right_thumb.y * frame_shape_0), right_thumb.z],
                [int(right_wrist.x * frame_shape_1), int(right_wrist.y * frame_shape_0), right_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            right_center_3d = np.mean(right_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            right_distances = [np.linalg.norm([p[0] - right_center_3d[0], p[1] - right_center_3d[1], p[2] - right_center_3d[2]]) for p in right_hand_pts]
            radius_3d = int(sum(right_distances ) / len(right_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(right_center_3d[0]), int(right_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # DISPLAY AREA OF LEFT HAND ---------------
            left_hand_pts = np.array([
                [int(left_pinky.x * frame_shape_1), int(left_pinky.y * frame_shape_0), left_pinky.z],
                [int(left_index.x * frame_shape_1), int(left_index.y * frame_shape_0), left_index.z],
                [int(left_thumb.x * frame_shape_1), int(left_thumb.y * frame_shape_0), left_thumb.z],
                [int(left_wrist.x * frame_shape_1), int(left_wrist.y * frame_shape_0), left_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            left_center_3d = np.mean(left_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            left_distances = [np.linalg.norm([p[0] - left_center_3d[0], p[1] - left_center_3d[1], p[2] - left_center_3d[2]]) for p in left_hand_pts]
            radius_3d = int(sum(left_distances ) / len(left_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(left_center_3d[0]), int(left_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
            
            for hand_landmarks in hand_results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


SHOULDERS
Left_Shoulder --- x: 0.6006247401237488y: 0.6842455863952637z: -0.7260433435440063
Right_Shoulder --- x: 0.41879305243492126y: 0.7798696756362915z: -0.7390284538269043
ELBOWS
Left_elbow --- x: 0.6757476329803467y: 0.8005616664886475z: -0.7217869162559509
Right_elbow --- x: 0.38869112730026245y: 0.992190957069397z: -0.6776244044303894
LEFT HAND
Left_pinky --- x: 0.6095274686813354y: 0.6453037261962891z: -0.7754067182540894
left_index --- x: 0.5977755784988403y: 0.6239674091339111z: -0.7367778420448303
left_thumb --- x: 0.5976038575172424y: 0.647662341594696z: -0.7433553338050842
Left_wrist --- x: 0.6234422922134399y: 0.6819645166397095z: -0.7614826560020447
RIGHT HAND
Right_pinky --- x: 0.38789892196655273y: 1.2666548490524292z: -0.7062239646911621
Right_index --- x: 0.4104711413383484y: 1.2490664720535278z: -0.715764045715332
Right_thumb --- x: 0.42304667830467224y: 1.2259559631347656z: -0.6970827579498291
Right_wrist: x: 0.3971032500267029y: 1.2012895345687866z: -0.696501672

In [1]:
# JUST THE POSE
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            # Upper body coordinates
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            print("SHOULDERS")
            print("Left_Shoulder --- " + "x: " + str(left_shoulder.x) + "y: " + str(left_shoulder.y) +  "z: " + str(left_shoulder.z))
            print("Right_Shoulder --- " + "x: " + str(right_shoulder.x) + "y: " + str(right_shoulder.y) + "z: " + str(right_shoulder.z))

            print("ELBOWS")
            left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
            right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
            print("Left_elbow --- " +  "x: " + str(left_elbow.x) + "y: " + str(left_elbow.y) + "z: " + str(left_elbow.z))
            print("Right_elbow --- " + "x: " + str(right_elbow.x) + "y: " + str(right_elbow.y) + "z: " + str(right_elbow.z))

            # LEFT HAND
            left_pinky = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value]
            left_index = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value]
            left_thumb = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value]
            left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
            print("LEFT HAND")
            print("Left_pinky --- " + "x: " + str(left_pinky.x) + "y: " + str(left_pinky.y) + "z: " + str(left_pinky.z))
            print("left_index --- " + "x: " + str(left_index.x) + "y: " + str(left_index.y) + "z: " + str(left_index.z))
            print("left_thumb --- " +  "x: " + str(left_thumb.x) + "y: " + str(left_thumb.y) + "z: " + str(left_thumb.z))
            print("Left_wrist ---" + "x: " + str(left_wrist.x) + "y: " + str(left_wrist.y) + "z: " + str(left_wrist.z))

            # RIGHT HAND
            right_pinky = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value]
            right_index = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value]
            right_thumb = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value]
            right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
            print("RIGHT HAND")
            print("Right_pinky --- " + "x: " + str(right_pinky.x) + "y: " + str(right_pinky.y) + "z: " + str(right_pinky.z))
            print("Right_index --- " + "x: " + str(right_index.x) + "y: " + str(right_index.y) + "z: " + str(right_index.z))
            print("Right_thumb --- " + "x: " + str(right_thumb.x) + "y: " + str(right_thumb.y) + "z: " + str(right_thumb.z))
            print("Right_wrist:" + "x: " + str(right_wrist.x) + "y: " + str(right_wrist.y) + "z: " + str(right_wrist.z))

            # Lower body coordinates
            left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
            print("LEFT KNEE --- "+ " x: " + str(left_knee.x) + " y: " + str(left_knee.y) + "z: " + str(left_knee.z))

            right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
            print("RIGHT KNEE --- "+ " x: " + str(right_knee.x) + " y: " + str(right_knee.y) + "z: " + str(right_knee.z))

            # LEFT FOOT
            left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
            left_heel = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value]
            left_foot_index = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value]
            print("LEFT FOOT")
            print("Left_ankle --- " + " x: " + str(left_ankle.x) + " y: " + str(left_ankle.y) + " z: " + str(left_ankle.z))
            print("Left_heel --- " + " x: " + str(left_heel.x) + " y: " + str(left_heel.y) + " z: " + str(left_heel.z))
            print("Left_foot_index --- " + " x: " + str(left_foot_index.x) + " y: " + str(left_foot_index.y) + " z: " + str(left_foot_index.z))

            # RIGHT FOOT
            right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]
            right_heel = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value]
            right_foot_index = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value]
            print("RIGHT FOOT")
            print("Right_ankle --- " + "x: " + str(right_ankle.x) + " y: " + str(right_ankle.y) + " z: " + str(right_ankle.z))
            print("Right_heel --- " + "x: " + str(right_heel.x) + " y: " + str(right_heel.y) + " z: " + str(right_heel.z))
            print("Right_foot_index --- " + "x: " + str(right_foot_index.x) + " y: " + str(right_foot_index.y) + " z: " + str(right_foot_index.z))

            # DISPLAY AREA OF RIGHT FOOT ---------------
            right_foot_pts = np.array([
                [int(right_ankle.x * frame.shape[1]), int(right_ankle.y * frame.shape[0])],
                [int(right_heel.x * frame.shape[1]), int(right_heel.y * frame.shape[0])],
                [int(right_foot_index.x * frame.shape[1]), int(right_foot_index.y * frame.shape[0])]
            ], np.int32)
            right_foot_pts = right_foot_pts.reshape((-1, 1, 2))

            # Fill the area of the right foot with a specific color
            cv2.fillPoly(image, [right_foot_pts], (245, 117, 66))

            # DISPLAY AREA OF LEFT FOOT ---------------
            left_foot_pts = np.array([
                [int(left_ankle.x * frame.shape[1]), int(left_ankle.y * frame.shape[0])],
                [int(left_heel.x * frame.shape[1]), int(left_heel.y * frame.shape[0])],
                [int(left_foot_index.x * frame.shape[1]), int(left_foot_index.y * frame.shape[0])]
            ], np.int32)
            left_foot_pts = left_foot_pts.reshape((-1, 1, 2))

            # Fill the area of the left foot with a specific color
            cv2.fillPoly(image, [left_foot_pts], (245, 117, 66))
    
            # DISPLAY AREA OF RIGHT HAND ---------------
            right_hand_pts = np.array([
                [int(right_pinky.x * frame.shape[1]), int(right_pinky.y * frame.shape[0]), right_pinky.z],
                [int(right_index.x * frame.shape[1]), int(right_index.y * frame.shape[0]), right_index.z],
                [int(right_thumb.x * frame.shape[1]), int(right_thumb.y * frame.shape[0]), right_thumb.z],
                [int(right_wrist.x * frame.shape[1]), int(right_wrist.y * frame.shape[0]), right_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            right_center_3d = np.mean(right_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            right_distances = [np.linalg.norm([p[0] - right_center_3d[0], p[1] - right_center_3d[1], p[2] - right_center_3d[2]]) for p in right_hand_pts]
            radius_3d = int(sum(right_distances ) / len(right_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(right_center_3d[0]), int(right_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # DISPLAY AREA OF LEFT HAND ---------------
            left_hand_pts = np.array([
                [int(left_pinky.x * frame.shape[1]), int(left_pinky.y * frame.shape[0]), left_pinky.z],
                [int(left_index.x * frame.shape[1]), int(left_index.y * frame.shape[0]), left_index.z],
                [int(left_thumb.x * frame.shape[1]), int(left_thumb.y * frame.shape[0]), left_thumb.z],
                [int(left_wrist.x * frame.shape[1]), int(left_wrist.y * frame.shape[0]), left_wrist.z]
            ], np.int32)

            # Calculate the center of the circle in 3D space (x, y, z)
            left_center_3d = np.mean(left_hand_pts, axis=0)

            # Calculate the radius of the circle in 3D space based on the average distance from the center to each point
            left_distances = [np.linalg.norm([p[0] - left_center_3d[0], p[1] - left_center_3d[1], p[2] - left_center_3d[2]]) for p in left_hand_pts]
            radius_3d = int(sum(left_distances ) / len(left_distances))

            # Draw the circle in 3D space
            cv2.circle(image, (int(left_center_3d[0]), int(left_center_3d[1])), radius_3d, (245, 117, 66), thickness=-1)  # -1 thickness for a filled circle

            # Calculate angle
            # angle = calculate_angle(shoulder, elbow, wrist)
            # Visualize angle
            # cv2.putText(image, str(angle),
            #                tuple(np.multiply(elbow, [640, 480]).astype(int)),
            #                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            #                     )

        except:
            pass

        # print(results.pose_landmarks)
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        cv2.imshow('Pose Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined